## Udacity course - Deep Learning
### Assignment 4

The goal of this assignment is make the neural network convolutional.

(Problems and sample code are taken from Udacity's free course on Deep Learning).
<br>
First the necessary packages are imported. Then the notMNIST dataset is loaded (containing characters A to J).


In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import os

In [3]:
# IMPORT notMNIST dataset
data_root = "..." # Change to actual folder

pickle_file = os.path.join(data_root, 'notMNIST.pickle')

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [4]:
#Reformat data + hot encoding
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [12]:
#test first convolutional model
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [13]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.181934
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.224262
Minibatch accuracy: 62.5%
Validation accuracy: 45.4%
Minibatch loss at step 100: 0.603860
Minibatch accuracy: 81.2%
Validation accuracy: 65.9%
Minibatch loss at step 150: 0.985842
Minibatch accuracy: 75.0%
Validation accuracy: 73.7%
Minibatch loss at step 200: 1.006194
Minibatch accuracy: 68.8%
Validation accuracy: 77.7%
Minibatch loss at step 250: 0.784734
Minibatch accuracy: 68.8%
Validation accuracy: 75.2%
Minibatch loss at step 300: 0.350058
Minibatch accuracy: 87.5%
Validation accuracy: 79.0%
Minibatch loss at step 350: 1.176797
Minibatch accuracy: 68.8%
Validation accuracy: 78.8%
Minibatch loss at step 400: 0.614065
Minibatch accuracy: 87.5%
Validation accuracy: 79.8%
Minibatch loss at step 450: 0.350685
Minibatch accuracy: 87.5%
Validation accuracy: 79.6%
Minibatch loss at step 500: 0.441691
Minibatch accuracy: 87.5%
Validation accuracy: 80.9%
M

## Problem 1

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (nn.max_pool()) of stride 2 and kernel size 2.

In [15]:
# EXERCISE 1 - add max-pooling
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # CNN weights
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden1 = tf.nn.relu(conv1 + layer1_biases)
        pool1 = tf.nn.max_pool(hidden1, [1,2,2,1], [1,2,2,1], padding='SAME')
        conv2 = tf.nn.conv2d(pool1, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden2 = tf.nn.relu(conv2 + layer2_biases)
        pool2 = tf.nn.max_pool(hidden2, [1,2,2,1], [1,2,2,1], padding='SAME')
        shape = pool2.get_shape().as_list()
        reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [16]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.740667
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.450797
Minibatch accuracy: 56.2%
Validation accuracy: 48.3%
Minibatch loss at step 100: 0.874351
Minibatch accuracy: 81.2%
Validation accuracy: 64.0%
Minibatch loss at step 150: 1.117167
Minibatch accuracy: 68.8%
Validation accuracy: 69.0%
Minibatch loss at step 200: 1.318190
Minibatch accuracy: 75.0%
Validation accuracy: 77.8%
Minibatch loss at step 250: 0.591473
Minibatch accuracy: 68.8%
Validation accuracy: 76.0%
Minibatch loss at step 300: 0.254064
Minibatch accuracy: 93.8%
Validation accuracy: 80.0%
Minibatch loss at step 350: 1.076521
Minibatch accuracy: 68.8%
Validation accuracy: 80.3%
Minibatch loss at step 400: 0.581914
Minibatch accuracy: 81.2%
Validation accuracy: 79.8%
Minibatch loss at step 450: 0.417091
Minibatch accuracy: 87.5%
Validation accuracy: 81.6%
Minibatch loss at step 500: 0.343211
Minibatch accuracy: 87.5%
Validation accuracy: 81.6%
M

## Problem 2

Try to get the best performance you can using a convolutional net. Look for example at the classic LeNet5 architecture, adding Dropout, and/or adding learning rate decay.

<br>
The LeNet5 has 6 hidden layers. Using the coding style as in the example given above quickly becomes a bit cumbersome. So I will again switch to the tensorflow `nn.layers.`.

In [11]:
# EXERCISE 2 - LeNet-5

image_size = 28
num_labels = 10
num_channels = 1 # grayscale

learning_rate = 0.05

batch_size = 32

graph = tf.Graph()

with graph.as_default():
   
    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(None, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # DEFINE CNN MODEL:
    def CNN_model(data):

        # L1: Convolutional Layer
        conv1 = tf.layers.conv2d(
            inputs=data, #input_layer,
            filters=6,
            kernel_size=[5, 5],
            padding="same",
            activation=tf.nn.relu)

        # L2: Pooling
        pool2 = tf.layers.average_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

        # L3: Convolutional Layer 
        # + L4: Pooling
        conv3 = tf.layers.conv2d(
            inputs=pool2,
            filters=16,
            kernel_size=[5, 5],
            padding="valid",
            activation=tf.nn.relu)
        pool4 = tf.layers.average_pooling2d(inputs=conv3, pool_size=[2, 2], strides=2)

        # L5: Convolutional Layer
        conv5 = tf.layers.conv2d(
            inputs=pool4,
            filters=120,
            kernel_size=[5, 5],
            padding="valid",
            activation=tf.nn.relu)

        # L6: Dense Layer 
        pool2_flat = tf.reshape(conv5, [-1, 120])
        dense6 = tf.layers.dense(inputs=pool2_flat, units=84, activation=tf.nn.relu)

        return tf.layers.dense(inputs=dense6, units=10)
        
    # Training computation.
    logits = CNN_model(tf_train_dataset)

    # Calculate Loss (for both TRAIN and EVAL modes)
    #loss = tf.losses.sparse_softmax_cross_entropy(labels=tf_train_labels, logits=logits)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
    
    # Evaluate
    correct = tf.equal(tf.argmax(logits, 1), tf.argmax(tf_train_labels, 1))
    accuracy_measure = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))
    
    # Save model in early stopping
    model_saver = tf.train.Saver()
    

I'll also again switch to running epoches rather than only steps. <br>
And I added early stopping. This way I can also interrupt if things are too slow and restore the model afterwards.

In [22]:
# initiate early stopping
max_checks_without_progress = 5
checks_without_progress = 0
best_loss = np.infty

num_epochs = 50 #one epoch = 1x forward pass + backward pass of all training examples
num_batches = train_labels.shape[0] // batch_size

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    
    print("Initialized")
    
    for epoch in range(num_epochs):
        for step in range(num_batches):
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            
            # Generate a minibatch:
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
      
            session.run(training_op, feed_dict={tf_train_dataset : batch_data, tf_train_labels : batch_labels})
            
        #if (step % 100 == 0):
        feed_dict={tf_train_dataset: valid_dataset, tf_train_labels: valid_labels}
        loss_valid, acc_valid = session.run([loss, accuracy_measure], feed_dict)
        if loss_valid < best_loss:
            save_path = model_saver.save(session, "...\\Udacity_Deep_Learning\\notMNIST_CNN_model_01.ckpt")
            best_loss = loss_valid
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            (epoch+1), loss_valid, best_loss, acc_valid * 100))

    acc_test = accuracy_measure.eval(session = session, feed_dict={tf_train_dataset: test_dataset, tf_train_labels: test_labels})
    print("Test accuracy: ", acc_test)

Initialized
1	Validation loss: 0.375470	Best loss: 0.375470	Accuracy: 88.38%
2	Validation loss: 0.335501	Best loss: 0.335501	Accuracy: 89.87%
3	Validation loss: 0.317784	Best loss: 0.317784	Accuracy: 90.69%
4	Validation loss: 0.310961	Best loss: 0.310961	Accuracy: 90.83%
5	Validation loss: 0.306396	Best loss: 0.306396	Accuracy: 91.12%
6	Validation loss: 0.305763	Best loss: 0.305763	Accuracy: 91.08%
7	Validation loss: 0.302738	Best loss: 0.302738	Accuracy: 91.19%
8	Validation loss: 0.310203	Best loss: 0.302738	Accuracy: 91.13%
9	Validation loss: 0.313102	Best loss: 0.302738	Accuracy: 90.91%
10	Validation loss: 0.316214	Best loss: 0.302738	Accuracy: 90.86%
11	Validation loss: 0.321004	Best loss: 0.302738	Accuracy: 90.97%
12	Validation loss: 0.327168	Best loss: 0.302738	Accuracy: 90.84%
Early stopping!
Test accuracy:  0.9636


Fairly OK for a first try, > 96% accuracy or < 4 % error. Especially since it does not contain any fancier optimizations, such as momentum or Adam optimizer. 
<br>
Soon I'll try some more complex models to see how far I can get on the notMINST dataset.